# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
%matplotlib inline
%config Inline.Backend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from collections import OrderedDict
from PIL import Image
import numpy as np
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
validation_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True) 
validationloader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True) 
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
#import json moved to first import

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [5]:
# TODO: Build and train your network
#1) Load a pretrained Network
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:04<00:00, 129134425.48it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
# Freeze parameters 
for param in model.parameters():
    param.requires_grad = False
    
# change classifier:
# hyperparameters for the classifier
input_size = 25088
hidden_sizes = [5024, 1024, 256]
output_size = 102
dropout = 0.4

classifier = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('dropout1', nn.Dropout(dropout)),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('dropout2', nn.Dropout(dropout)),
                      ('fc3', nn.Linear(hidden_sizes[1], hidden_sizes[2])),
                      ('relu3', nn.ReLU()),
                      ('dropout3', nn.Dropout(dropout)),
                      ('output', nn.Linear(hidden_sizes[2], output_size)),
                      ('softmax', nn.LogSoftmax(dim=1))
]))
model.classifier = classifier
# Check
model.classifier

Sequential(
  (fc1): Linear(in_features=25088, out_features=5024, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.4)
  (fc2): Linear(in_features=5024, out_features=1024, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=1024, out_features=256, bias=True)
  (relu3): ReLU()
  (output): Linear(in_features=256, out_features=102, bias=True)
  (softmax): LogSoftmax()
)

In [7]:
#criterion
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [8]:
# function for the validation pass
def validation(model, validationloader, criterion):
    validation_loss = 0
    accuracy = 0
    
    for images, labels in validationloader:
        images, labels = images.to(device), labels.to(device)
        
        output = model.forward(images)
        validation_loss = criterion(output, labels).item()
       
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return validation_loss, accuracy        
        

In [9]:
# training and printing the accuracy
from workspace_utils import active_session
epochs = 20
steps = 0
running_loss = 0
print_every = 40
print('training started')
with active_session():
    for e in range(epochs):
        model.to(device)
        for images, labels in trainloader:
            steps += 1
            images, labels = images.to(device), labels.to(device)


            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    validation_loss, accuracy = validation(model, validationloader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Valid Loss: {:.3f}.. ".format(validation_loss/len(validationloader)),
                      "Validation Accuracy: {:.3f}".format(accuracy/len(validationloader)))

                running_loss = 0

                # Make sure training is back on
            model.train()
print('training vgg model finished')

training started
Epoch: 1/20..  Training Loss: 4.392..  Valid Loss: 0.258..  Validation Accuracy: 0.197
Epoch: 1/20..  Training Loss: 3.153..  Valid Loss: 0.179..  Validation Accuracy: 0.365
Epoch: 2/20..  Training Loss: 2.396..  Valid Loss: 0.113..  Validation Accuracy: 0.512
Epoch: 2/20..  Training Loss: 2.029..  Valid Loss: 0.105..  Validation Accuracy: 0.585
Epoch: 2/20..  Training Loss: 1.841..  Valid Loss: 0.077..  Validation Accuracy: 0.581
Epoch: 3/20..  Training Loss: 1.669..  Valid Loss: 0.056..  Validation Accuracy: 0.681
Epoch: 3/20..  Training Loss: 1.537..  Valid Loss: 0.066..  Validation Accuracy: 0.693
Epoch: 4/20..  Training Loss: 1.462..  Valid Loss: 0.100..  Validation Accuracy: 0.685
Epoch: 4/20..  Training Loss: 1.379..  Valid Loss: 0.064..  Validation Accuracy: 0.729
Epoch: 4/20..  Training Loss: 1.297..  Valid Loss: 0.079..  Validation Accuracy: 0.750
Epoch: 5/20..  Training Loss: 1.253..  Valid Loss: 0.139..  Validation Accuracy: 0.765
Epoch: 5/20..  Training Lo

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [10]:
# TODO: Do validation on the test set

with torch.no_grad():
    test_loss, accuracy = validation(model, testloader, criterion)

print("Test-Loss: {}\n".format(test_loss/len(testloader)),
     "Test-Accuracy: {}".format(accuracy/len(testloader)))

Test-Loss: 0.0782383130146907
 Test-Accuracy: 0.8307645320892334


In [11]:
# TODO: Save the checkpoint 
checkpoint = {
    'arch': 'vgg16',
    'input_size': model.classifier[0].in_features,
    'state_dict': model.classifier.state_dict(),
    'class_to_idx': train_data.class_to_idx,
    'output_size': output_size,
    'classifier': model.classifier,
    'hidden_layers': hidden_sizes,
    'dropout': dropout,
    'optimizer state': optimizer.state_dict,
    'number of epochs': epochs,
}

torch.save(checkpoint, 'checkpoint-jupyter.pth')
print('saved checkpoint')

saved checkpoint


## testing a second model

In [12]:
# TODO: Build and train your network
#1) Load a pretrained Network
model2 = models.densenet121(pretrained=True)
model2

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 84164923.06it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [13]:
# Freeze parameters 
for param in model2.parameters():
    param.requires_grad = False
    
# change classifier:
# hyperparameters for the classifier
input_size = 1024
hidden_sizes = [512, 256, 256]
output_size = 102
dropout = 0.5

classifier = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('dropout1', nn.Dropout(dropout)),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('dropout1', nn.Dropout(dropout)),
                      ('fc3', nn.Linear(hidden_sizes[1], hidden_sizes[2])),
                      ('relu3', nn.ReLU()),
                      ('dropout1', nn.Dropout(dropout)),
                      ('output', nn.Linear(hidden_sizes[2], output_size)),
                      ('softmax', nn.LogSoftmax(dim=1))
]))
model2.classifier = classifier
# Check
model2.classifier

Sequential(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (relu3): ReLU()
  (output): Linear(in_features=256, out_features=102, bias=True)
  (softmax): LogSoftmax()
)

In [14]:
criterion = nn.NLLLoss()
optimizer2 = optim.Adam(model2.classifier.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

remark: validation function can be used from above as it takes the model as an input paramter

In [15]:
from workspace_utils import active_session
# training and printing the accuracy
epochs = 20
steps = 0
running_loss = 0
print_every = 40
print('training started')
with active_session():
    for e in range(epochs):
        model2.to(device)
        for images, labels in trainloader:
            steps += 1
            images, labels = images.to(device), labels.to(device)


            optimizer2.zero_grad()

            output = model2.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer2.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    validation_loss, accuracy = validation(model2, validationloader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Valid Loss: {:.3f}.. ".format(validation_loss/len(validationloader)),
                      "Validation Accuracy: {:.3f}".format(accuracy/len(validationloader)))

                running_loss = 0

                # Make sure training is back on
                model2.train()
print('training finished')

training started
Epoch: 1/20..  Training Loss: 4.507..  Valid Loss: 0.343..  Validation Accuracy: 0.059
Epoch: 1/20..  Training Loss: 4.048..  Valid Loss: 0.277..  Validation Accuracy: 0.199
Epoch: 2/20..  Training Loss: 3.198..  Valid Loss: 0.229..  Validation Accuracy: 0.299
Epoch: 2/20..  Training Loss: 2.702..  Valid Loss: 0.161..  Validation Accuracy: 0.396
Epoch: 2/20..  Training Loss: 2.312..  Valid Loss: 0.147..  Validation Accuracy: 0.492
Epoch: 3/20..  Training Loss: 2.164..  Valid Loss: 0.135..  Validation Accuracy: 0.545
Epoch: 3/20..  Training Loss: 1.898..  Valid Loss: 0.125..  Validation Accuracy: 0.531
Epoch: 4/20..  Training Loss: 1.814..  Valid Loss: 0.119..  Validation Accuracy: 0.576
Epoch: 4/20..  Training Loss: 1.650..  Valid Loss: 0.111..  Validation Accuracy: 0.619
Epoch: 4/20..  Training Loss: 1.692..  Valid Loss: 0.090..  Validation Accuracy: 0.626
Epoch: 5/20..  Training Loss: 1.547..  Valid Loss: 0.118..  Validation Accuracy: 0.640
Epoch: 5/20..  Training Lo

In [ ]:
# TODO: Do validation of model 2 on the test set

with torch.no_grad():
    test_loss, accuracy = validation(model2, testloader, criterion)

print("Test-Loss: {}\n".format(test_loss/len(testloader)),
     "Test-Accuracy: {}".format(accuracy/len(testloader)))

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {
    'arch': 'densenet121',
    'input_size': model2.classifier[0].in_features,
    'state_dict': model2.classifier.state_dict(),
    'class_to_idx': train_data.class_to_idx,
    'output_size': output_size,
    'classifier': model2.classifier,
    'hidden_layers': hidden_sizes,
    'dropout': dropout,
    'optimizer state': optimizer2.state_dict,
    'number of epochs': epochs,
}

torch.save(checkpoint, 'checkpoint-jupyter_model2.pth')
print('saved checkpoint')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {
    'arch': 'vgg16',
    'input_size': model.classifier[0].in_features,
    'state_dict': model.classifier.state_dict(),
    'class_to_idx': train_data.class_to_idx,
    'output_size': output_size,
    'classifier': model.classifier,
    'hidden_layers': hidden_sizes,
    'dropout': dropout,
    'optimizer state': optimizer.state_dict,
    'number of epochs': epochs,
}

torch.save(checkpoint, 'checkpoint-jupyterm1.pth')
print('saved checkpoint')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    #arch = checkpoint['arch']
    model = models.vgg16(pretrained=True)
    classifier = checkpoint['classifier']
    model.classifier = classifier
    model.classifier.load_state_dict(checkpoint['state_dict'])
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model, checkpoint['class_to_idx']
    
model, class_to_idx = load_checkpoint('checkpoint-jupyter.pth')
print('model is loaded')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Open the image and resize it
    image_pil = Image.open(image)
    image_pil = image_pil.resize((256,256))
    
    # Resize - Adapted from https://stackoverflow.com/questions/16646183/crop-an-image-in-the-centre-using-pil
    width, height = image_pil.size   # Get dimensions

    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2

    image_pil = image_pil.crop((left, top, right, bottom))
    
    # Turn into a np_array
    np_image = np.array(image_pil)/255
    
    # Undo mean and std and then transpose the array
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])  
    np_image = (np_image - mean)/std
    np_image = np.transpose(np_image, (2, 0, 1))
    
    return np_image
    

In [ ]:
# Test process_image implementation with a random picture from the test set
test = process_image('flowers/test/11/image_03115.jpg')
print(test.shape)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(test)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Inverting class_to_idx - Kudos to Mentor ShintoT
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    model= model.to(device)
    img_predict = torch.FloatTensor([process_image(image_path)])
    img_predict = img_predict.to(device)

    with torch.no_grad():
        result = model(img_predict)
    
    
    # Save probs and classes
    probs = np.array(torch.exp(result).topk(topk)[0])
    classes = np.array(torch.exp(result).topk(topk)[1])
    classes = [idx_to_class[c] for c in classes[0,:]]  
    
    return(probs, classes)
    


In [ ]:
#testing it
print(predict('flowers/test/11/image_03115.jpg', model))

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def sanity_check(pathToImage, model):
    #process image
    processed_image = process_image(pathToImage)
    #get the probability and the top classes for the image
    probs, classes = predict(pathToImage, model)
    #create subplot
    fig, (p1,p2) = plt.subplots(figsize=(10,10), ncols=1, nrows=2)                         
    #print(cat_to_name[classes[0]])
    p1.set_title(cat_to_name[classes[0]])                          
    # Show the image for reference
    p1.imshow(Image.open(pathToImage))
    p1.axis('off')
                  
    y_pos = np.arange(len(probs[0]))

    p2.barh(y_pos, probs[0])
    plt.yticks(y_pos)
    p2.set_yticklabels([cat_to_name[c] for c in classes])
    p2.invert_yaxis()
    p2.set_title("Class Probabilities")
                             

In [ ]:
sanity_check('flowers/test/85/image_04805.jpg', model)